In [2]:
from hashlib import md5
# from sklearn import tree
import pandas as pd
import numpy as np
import openpyxl
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression
from pathlib import Path
from datetime import datetime

In [3]:
# def MSE(y_true, y_pred):
#     """
#     Returns the Mean Squared Error
#     """
#     return ((y_true - y_pred) ** 2).mean()



In [4]:
def get_data():
    """
    Returns the data from the xlsx file
    """
    file_name = 'yeast.csv' 
    df = pd.read_csv(file_name, index_col=0, header=None)
    # print(df.head()) # print the first 5 rows
    return df

md = get_data()
md.dropna(inplace = True)
print(md.shape)
md.replace('?', 0, inplace = True)

print(md)

(1484, 9)
               1     2     3     4    5    6     7     8    9
0                                                            
ADT1_YEAST  0.58  0.61  0.47  0.13  0.5  0.0  0.48  0.22  MIT
ADT2_YEAST  0.43  0.67  0.48  0.27  0.5  0.0  0.53  0.22  MIT
ADT3_YEAST  0.64  0.62  0.49  0.15  0.5  0.0  0.53  0.22  MIT
AAR2_YEAST  0.58  0.44  0.57  0.13  0.5  0.0  0.54  0.22  NUC
AATM_YEAST  0.42  0.44  0.48  0.54  0.5  0.0  0.48  0.22  MIT
...          ...   ...   ...   ...  ...  ...   ...   ...  ...
YUR1_YEAST  0.81  0.62  0.43  0.17  0.5  0.0  0.53  0.22  ME2
ZIP1_YEAST  0.47  0.43  0.61  0.40  0.5  0.0  0.48  0.47  NUC
ZNRP_YEAST  0.67  0.57  0.36  0.19  0.5  0.0  0.56  0.22  ME2
ZUO1_YEAST  0.43  0.40  0.60  0.16  0.5  0.0  0.53  0.39  NUC
G6PD_YEAST  0.65  0.54  0.54  0.13  0.5  0.0  0.53  0.22  CYT

[1484 rows x 9 columns]


In [5]:
X = md.iloc[:, 0:8].values.reshape(-1, 8)


print(X)
print(X.shape)
# add 1 to the first column
X = np.insert(X, 0, 1, axis=1)
print('X:', X)
print(X.shape)

Y = md.iloc[:, 8:9].values.reshape(-1, 1)


[[0.58 0.61 0.47 ... 0.   0.48 0.22]
 [0.43 0.67 0.48 ... 0.   0.53 0.22]
 [0.64 0.62 0.49 ... 0.   0.53 0.22]
 ...
 [0.67 0.57 0.36 ... 0.   0.56 0.22]
 [0.43 0.4  0.6  ... 0.   0.53 0.39]
 [0.65 0.54 0.54 ... 0.   0.53 0.22]]
(1484, 8)
X: [[1.   0.58 0.61 ... 0.   0.48 0.22]
 [1.   0.43 0.67 ... 0.   0.53 0.22]
 [1.   0.64 0.62 ... 0.   0.53 0.22]
 ...
 [1.   0.67 0.57 ... 0.   0.56 0.22]
 [1.   0.43 0.4  ... 0.   0.53 0.39]
 [1.   0.65 0.54 ... 0.   0.53 0.22]]
(1484, 9)


In stochastic (or "on-line") gradient descent, the true gradient of
*Q*(*w*) is approximated by a gradient at a single sample:

  
*w* := *w* − *η*∇*Q*<sub>*i*</sub>(*w*).

As the algorithm sweeps through the training set, it performs the above
update for each training sample. Several passes can be made over the
training set until the algorithm converges. If this is done, the data
can be shuffled for each pass to prevent cycles. Typical implementations
may use an [adaptive learning rate] so that the algorithm converges.[1]

In pseudocode, stochastic gradient descent can be presented as :

<div style="margin-left: 35px; width: 600px">
-   Choose an initial vector of parameters *w* and learning rate *η*.
-   Repeat until an approximate minimum is obtained:
    -   Randomly shuffle samples in the training set.
    -   For *i* = 1, 2, ..., *n*, do:
        -   *w* := *w* − *η*∇*Q*<sub>*i*</sub>(*w*).


  [adaptive learning rate]: adaptive_learning_rate "wikilink"

In [6]:
def p(i):
    return 1 / (1 + np.exp(-i))

In [32]:
def pi(x, beta):
    return np.exp(np.dot(x, beta)) / (1 + np.exp(np.dot(x, beta)))

In [7]:
outputs = ['CYT', 'NUC', 'MIT', 'ME3', 'ME2', 'ME1', 'EXC', 'VAC', 'POX', 'ERL']

y = np.zeros((len(Y), 10))
print(y.shape)

for i in range(len(Y)):
    for j in range(10):
        if outputs[j] == Y[i]:
            y[i][j] = 1
        else:
            y[i][j] = 0

print('results', Y[0] == outputs[2])
print(y.shape)
# for i in range(10):
#     print(outputs[i], ': ', y[i])

(1484, 10)
results [ True]
(1484, 10)


In [27]:
eta = 0.01
beta = np.zeros((10, 9))
trainNo = 1000


gradient = np.zeros((10, 9))

# gradient[0] = 9999

# stochastic gradient descent
n = 0
while (n < 1000):
#while (np.linalg.norm(gradient) > 0.01):
    # gradient = np.zeros(9)

    # take one sample randomly
    i = np.random.randint(0, trainNo)
    print(i)
    
    # calculate the gradient
    for k in range(10):
        for j in range(9):
            gradient[k][j] = (p(np.dot(X[i], beta[k])) - y[i][k]) * X[i][j]

    print('grad', gradient)

    # update beta
    for k in range (10):
        beta[k] = beta[k] - eta * gradient[k]
    
    n = n + 1

40
grad [[ 0.5    0.43   0.34   0.195  0.12   0.25   0.     0.27   0.11 ]
 [ 0.5    0.43   0.34   0.195  0.12   0.25   0.     0.27   0.11 ]
 [ 0.5    0.43   0.34   0.195  0.12   0.25   0.     0.27   0.11 ]
 [ 0.5    0.43   0.34   0.195  0.12   0.25   0.     0.27   0.11 ]
 [-0.5   -0.43  -0.34  -0.195 -0.12  -0.25  -0.    -0.27  -0.11 ]
 [ 0.5    0.43   0.34   0.195  0.12   0.25   0.     0.27   0.11 ]
 [ 0.5    0.43   0.34   0.195  0.12   0.25   0.     0.27   0.11 ]
 [ 0.5    0.43   0.34   0.195  0.12   0.25   0.     0.27   0.11 ]
 [ 0.5    0.43   0.34   0.195  0.12   0.25   0.     0.27   0.11 ]
 [ 0.5    0.43   0.34   0.195  0.12   0.25   0.     0.27   0.11 ]]
238
grad [[ 0.49673467  0.26326938  0.21856326  0.25333468  0.16392244  0.24836734
   0.          0.25830203  0.10928163]
 [ 0.49673467  0.26326938  0.21856326  0.25333468  0.16392244  0.24836734
   0.          0.25830203  0.10928163]
 [-0.50326533 -0.26673062 -0.22143674 -0.25666532 -0.16607756 -0.25163266
  -0.         -0.26169

In [35]:
# one or not classification results
x_pred = np.zeros(len(X) - trainNo)
estimates = np.zeros(10)
for i in range(len(x_pred)):
    for k in range(10):
        estimates[k] = pi(X[i + trainNo], beta[k])
    x_pred[i] = np.argmax(estimates)
    print(estimates)
    print(x_pred[i])
print(x_pred)

[0.24639253 0.26590012 0.15876155 0.0634947  0.04833664 0.03648888
 0.04620577 0.03109308 0.0342441  0.02974028]
1.0
[0.27509363 0.30624731 0.19422524 0.09030871 0.06917042 0.0549886
 0.0664916  0.04892725 0.0530695  0.04696318]
1.0
[0.27574413 0.30184484 0.19065233 0.08851914 0.06814867 0.05408699
 0.06582582 0.04785578 0.05197326 0.04597652]
1.0
[0.27019903 0.29881679 0.18047582 0.08328446 0.06397955 0.0501653
 0.06147647 0.04407549 0.04799745 0.04229729]
1.0
[0.26656213 0.29667653 0.1973095  0.08664434 0.06684445 0.05304563
 0.06441363 0.0466767  0.05069348 0.04486909]
1.0
[0.29088741 0.32663365 0.20281997 0.10456012 0.08170994 0.06588484
 0.07846159 0.0593847  0.06405113 0.05716455]
1.0
[0.28374279 0.30987844 0.19204358 0.09127948 0.07131532 0.05655331
 0.06867854 0.05048513 0.05486083 0.04852197]
1.0
[0.28374279 0.30987844 0.19204358 0.09127948 0.07131532 0.05655331
 0.06867854 0.05048513 0.05486083 0.04852197]
1.0
[0.29782468 0.32431676 0.20730348 0.10659042 0.08319001 0.06754578

In [37]:

def pij(x, beta):

    sum = 0
    for l in range(9):
        sum = sum + np.exp(np.dot(X[i + trainNo], beta))


    return np.exp(np.dot(x, beta)) / (1 + sum)

    

In [42]:
for i in range(len(x_pred)):

    for k in range(10):
        print(pij(X[i + trainNo], beta[k]), end = ' ')


    print()


0.08292861013646141 0.0850281521131255 0.06993616216804543 0.042106420986063195 0.03485748673646448 0.028244114528071674 0.03373555487067955 0.02489947224895402 0.026880197180088716 0.024024352474357337 
0.4526405234423113
0.08594664106266403 0.08876789017405978 0.07605336889468202 0.05242978311825435 0.04452945359538095 0.03818894513314345 0.04340373320707266 0.03516358603572664 0.037253361463899654 0.03413752516049285 
0.5358742878453764
0.08601003428951141 0.08839419236321519 0.07549933517258914 0.0518215479136362 0.044103767780603986 0.0377519000984809 0.043119048431327425 0.03460672374062401 0.036709825237366475 0.03361318019779559 
0.5316295552251503
0.08546295928313505 0.08813265469698758 0.07385028913428769 0.04998240258823441 0.042319096332322476 0.035798545070010526 0.04120926934618361 0.03258565868476856 0.034680751840833204 0.031603385854001236 
0.5156250128307643
0.08509573280441239 0.08794553056499496 0.07652175209513114 0.05117331408604637 0.04355380871918307 0.037241596

In [28]:
len(X)

1484